# Import packages

In [1]:
from pathlib import Path
import os, sys
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break

%load_ext autoreload
%autoreload 2


from data_wrangling import specific_utils, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.cluster_replacement import cluster_replacement_utils
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_collect_info_class, GUAT_combine_info_class
from decision_making_analysis.compare_GUAT_and_TAFT import GUAT_vs_TAFT_class, GUAT_vs_TAFT_x_sessions_class, helper_GUAT_vs_TAFT_class
from visualization.matplotlib_tools import plot_trials, plot_behaviors_utils
from visualization.animation import animation_class
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curv_of_traj_utils
from machine_learning.ml_methods import regression_utils, classification_utils, prep_ml_data_utils, hyperparam_tuning_class
from visualization.plotly_polar_tools import plotly_utils_polar, plotly_for_ff_polar, plotly_for_trajectory_polar
from machine_learning.ml_methods import ml_methods_class
from visualization.dash_tools.dash_main_class_methods import dash_applied_to_GUAT_TAFT
from visualization.plotly_tools import plotly_preparation, plotly_for_time_series, plotly_for_null_arcs, plotly_for_correlation, plotly_for_monkey, plotly_plot_class

import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from os.path import exists
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import gc
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import warnings
import os, sys, sys
from importlib import reload
from sklearn.exceptions import ConvergenceWarning
from importlib import reload


plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 50



Set up logging configuration.


/opt/miniconda3/envs/multiff_clean/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


# Data for one session

In [2]:
reload(dash_applied_to_GUAT_TAFT)
dg = dash_applied_to_GUAT_TAFT.DashForGUATandTAFT(raw_data_folder_path='all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330')

opt_arc_type = 'norm_opt_arc'


# ref_point_params = {'ref_point_mode': 'distance',
#                     'ref_point_value': -150}

ref_point_params = {'ref_point_mode': 'distance',
                    'ref_point_value': -100}

curv_of_traj_params = {'curv_of_traj_mode': 'distance',
                        'window_for_curv_of_traj': [-25, 0],
                        'truncate_curv_of_traj_by_time_of_capture': False}

overall_params = {'eliminate_outliers': True,
                  'use_curv_to_ff_center': False,
                  'normalize': False,
                  'remove_i_o_modify_rows_with_big_ff_angles': True,
                  'change_units_to_degrees_per_m': True,
                  'heading_instead_of_curv': True}  

monkey_plot_params = {
    "show_visible_fireflies": True,
    "show_in_memory_fireflies": False,
    "show_monkey_heading": False,
    "show_visible_segments": True,
    "show_traj_portion": True,
    "show_null_arcs_to_ff": False,
    "show_extended_traj_arc": False,
    "show_stops": True,
    "show_cur_and_nxt_stops": True,
    "show_capture_stops": True,
    "show_all_eye_positions": False,
    "show_current_eye_positions": True,
    "show_eye_positions_for_both_eyes": True,
    "show_traj_color_as_speed":True,
    
}

time_series_plot_params = {
    "use_two_y_axes": False,
    "show_nxt_ff_curv_in_time_series_plot": True,
    "show_cur_ff_curv_in_time_series_plot": True,
    "show_d_curv_to_nxt_ff": True,
    "show_angle_to_nxt_ff": True
}

                     

dg.prepare_to_make_dash_for_main_plots(monkey_plot_params=monkey_plot_params,
                                        time_series_plot_params=time_series_plot_params,
                                        ref_point_params=ref_point_params,
                                        curv_of_traj_params=curv_of_traj_params,
                                        overall_params=overall_params,
                                        stop_point_index=126439 #46627
                                        )

#dg.plt.show()
#dg.fig_time_series_combd.show()


Retrieved monkey_information
The number of points that were removed due to delta_position exceeding the ceiling is 0
Note: ff_caught_T_sorted is replaced with ff_caught_T_new
Retrieved ff_dataframe from all_monkey_data/processed_data/monkey_Bruno/data_0330/ff_dataframe.h5
Retrieved stop_category_df
Percentage of control rows that have nxt ff cluster flashed on between stop_time - 2 and stop:             62.18 % of 156 rows.
Percentage of control rows that have nxt ff cluster flashed on between stop time and next stop:             94.87 % of 156 rows.
[Warning] 14.744% of ff_y_relative values are negative and will be removed.
[Warning] 12.782% of ff_x_relative > ff_y_relative. Rows will be removed.
[Warning] 0.862% of ff_x_relative > ff_y_relative. Rows will be removed.
Percentage of rows outside of [-45, 45]: 32.61%
2025-10-01 22:12:20,987 - WARNING - stop_point_index: 126439 is not in stops_near_ff_df. Using the first instance in stops_near_ff_df to plot.
duration_to_plot: [42.902454,

In [ ]:
from visualization.dash_tools import dash_utils

In [ ]:
reload(dash_utils)

In [ ]:
dg.monkey_plot_params['hide_non_essential_visible_segments']

In [ ]:
dg.make_dash_for_main_plots(show_trajectory_time_series=False, 
                            show_neural_plots=False,
                            port=8001)

In [12]:
dg.current_plotly_key_comp['trajectory_df']

,monkey_x,monkey_y,point_index,time,monkey_angle,speed,monkey_speeddummy,cum_distance,gaze_world_x,gaze_world_y,...,gaze_mky_view_x_l,gaze_mky_view_y_l,valid_view_point_l,gaze_world_x_r,gaze_world_y_r,gaze_mky_view_x_r,gaze_mky_view_y_r,valid_view_point_r,rel_distance,rel_time
0,669.11107,-163.96493,2590,42.91851,-1.73842,114.09157,1,2096.26665,-474.04992,456.70310,...,4.62404,64.36517,True,-472.43502,463.59740,3.74003,50.23095,True,-350.37000,-3.98000
1,668.39676,-162.41483,2591,42.93517,-1.73728,94.66694,1,2097.97342,-474.64412,453.32403,...,4.81984,66.88020,True,-472.80170,460.98758,3.72593,51.15439,True,-348.66000,-3.97000
2,667.78595,-161.10572,2592,42.95171,-1.73703,67.77767,1,2099.41801,-474.98842,451.31309,...,4.92339,68.20068,True,-472.95325,459.68053,3.67820,51.02297,True,-347.22000,-3.95000
3,667.45038,-160.39237,2593,42.96834,-1.73703,30.04542,1,2100.20635,-475.10506,450.24924,...,4.88542,68.36933,True,-473.07132,458.50387,3.60571,51.41462,True,-346.43000,-3.93000
4,667.36090,-160.20216,2594,42.98498,-1.73734,6.19756,1,2100.41655,-475.35050,448.73240,...,4.92836,72.16171,True,-472.83680,459.40540,3.50920,50.27763,True,-346.22000,-3.92000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497,668.63490,541.15871,3087,51.10213,-2.24002,85.37899,1,2895.75108,-879.15077,-105.02657,...,-4.51613,46.91686,True,-875.42421,-100.17418,-4.34104,34.68163,True,449.11000,4.20000
498,668.74793,542.75743,3088,51.11874,-2.24002,96.38261,1,2897.35379,-880.19927,-106.63020,...,-4.87506,47.86554,True,-876.21997,-101.15766,-4.32699,34.34395,True,450.72000,4.22000
499,668.86092,544.35502,3089,51.13537,-2.24001,101.15697,1,2898.95537,-882.16778,-109.34489,...,-5.12104,51.46358,True,-877.12436,-102.35626,-4.36114,34.24351,True,452.32000,4.23000
500,668.98525,546.11308,3090,51.15199,-2.24001,106.89148,1,2900.71782,-883.82889,-111.97588,...,-5.25441,51.77518,True,-879.26449,-105.90881,-4.88653,36.59503,True,454.08000,4.25000


In [5]:
dg.stop_point_index

49089

In [ ]:
dg._show_null_arcs_for_cur_and_nxt_ff_in_plotly()

In [ ]:
dg._produce_fig_for_dash()

In [ ]:
dg.fig

In [ ]:
dg._produce_initial_plots()

In [ ]:
dg.monkey_plot_params

In [ ]:
# assuming your figure is in dg.fig
trace_names = [trace.name for trace in dg.fig.data]
trace_names


In [ ]:
dg.fig.data

In [ ]:
dg._show_null_arcs_for_cur_and_nxt_ff_in_plotly()

In [ ]:
dg.fig = plotly_plot_class.PlotlyPlotter.make_one_monkey_plotly_plot(dg,
                                                                     monkey_plot_params=dg.monkey_plot_params)
dg.fig

# Appendix

## see class orders (for inheritance)

In [ ]:
dash_applied_to_GUAT_TAFT.DashForGUATandTAFT.mro()